# Домашнее задание-2. Грамматика составляющих

In [2]:
import re
import nltk
from collections import Counter
from nltk.corpus import conll2000
conll2000.ensure_loaded()

Для начала запишем вывод, который мы хотим получить для данных предложений:

Опишем разбор предложения "Этот веселый мальчик идёт" с помощью алгоритма Кока-Янгера-Касами:

In [30]:
import pandas as pd
data = [['S', '', '', ''], 
        ['NP', '', '', 'VP'], 
        ['Det', 'NP', '', 'VP'], 
        ['Det', 'ADJ', 'N', 'V'], 
        ['Этот', 'веселый', 'мальчик', 'идет']]
pd.DataFrame(data)

,0,1,2,3
0,S,,,
1,NP,,,VP
2,Det,NP,,VP
3,Det,ADJ,N,V
4,Этот,веселый,мальчик,идет


Правила сработали в такой последовательности:

1 Det -> Этот, ADJ -> веселый, N -> мальчик, V -> идет
2 NP -> ADJ N, VP -> V
3 NP -> Det NP
4 S -> NP VP

Запустим систему:

In [1]:
import nltk

In [26]:
rules = """
    S -> NP VP | VP | N VP
    NP -> Det N | ADJ N | NN | Det NP
    VP -> V NP | V VP | V
    Det -> 'Этот' | 'всякие' | 'мою' | 'какое-нибудь'
    ADJ -> 'веселый'
    N -> 'Вася' | 'книгу' | 'письмо' | 'мальчик' | 'книги'
    NN -> 'Он'
    V -> 'любит' | 'читать' | 'читает' | 'Напиши' | 'идет'
""".split('\n')

In [27]:
grammar = nltk.CFG.fromstring('\n'.join(rules))
cp = nltk.EarleyChartParser(grammar)

In [28]:
def print_parses(parser, sentence):
    for tree in parser.parse(nltk.word_tokenize(sentence)):
        print(tree, '\n')

Проверим действие системы на наших предложениях:

In [29]:
sentences = ["Вася читает мою книгу", "Напиши какое-нибудь письмо", "Этот веселый мальчик идет", "Он любит читать всякие книги"]
for sentence in sentences:
    print_parses(cp, sentence)

(S (N Вася) (VP (V читает) (NP (Det мою) (N книгу)))) 

(S (VP (V Напиши) (NP (Det какое-нибудь) (N письмо)))) 

(S (NP (Det Этот) (NP (ADJ веселый) (N мальчик))) (VP (V идет))) 

(S
  (NP (NN Он))
  (VP (V любит) (VP (V читать) (NP (Det всякие) (N книги))))) 



Опишем также возможную последовательность действий для работы с новыми словами. Так как все правила нашей грамматики на базовом уровне начинаются с определения части речи для слова, то, применив морфологический парсер типа pymorphy2 к новому тексту, для каждого разбора можно сопоставить стартовый элемент в нашей грамматике. Затем эти элементы будут связываться в структуру по правилам, которые необходимо полно описать.